# Capstone Week 3 Project - Part 1

Get neighborhood data for Toronto

In [19]:
# Import all needed packages here
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import lxml
from pprint import pprint
from collections import defaultdict

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# go get the web page
web_page = requests.get(url)
# make some tasty soup
soup = BeautifulSoup(web_page.content, 'html.parser')

I viewed the Wikipedia page in Firefox then pressed Ctrl+U to see the source. The data is in a table of class 'wikitable'. With this I will:
1. Get the HTML for that specific table
2. Parse out the row data
3. Load the data into a Python list
4. Create the pandas DataFrame from the list

**Note:** I am using ``defaultdict`` from the handy Python ``collections`` module to accumulate a list of neighborhoods in a borough. Then I do some data wrangling to transform that list into a comma separated string and output the results to the list for pandas DataFrame creation.

In [48]:
trono_fsas = soup.find('table', class_='wikitable')
html_rows = trono_fsas.find_all('tr')
data = []
d_pc_boro = defaultdict(list)
# loop thru the rows, populate dict with key (boro, pc), append list of neighborhoods
for tr in html_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if len(row) == 3:
        pc = row[0]
        boro = row[1]
        nbh = row[2]
        if boro == 'Not assigned':  # ignore cells with unassigned Borough
            continue
        if nbh == 'Not assigned':  # name unassigned neighborhoods to the name of the Borough
            nbh = boro
        d_pc_boro[(boro, pc)].append(nbh)  # group the neighborhoods
# loop thru the dict, turn list of neighborhoods to a string and output a list for pandas
for (bro, pcode), hoods in sorted(d_pc_boro.items()):
    nbstr = ', '.join(hoods)  # concatenate the hoods
    data.append([pcode, bro, nbstr])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df

,PostalCode,Borough,Neighborhood
0,M4N,Central Toronto,Lawrence Park
1,M4P,Central Toronto,Davisville North
2,M4R,Central Toronto,North Toronto West
3,M4S,Central Toronto,Davisville
4,M4T,Central Toronto,"Moore Park, Summerhill East"
5,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."
6,M5N,Central Toronto,Roselawn
7,M5P,Central Toronto,"Forest Hill North, Forest Hill West"
8,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
9,M4W,Downtown Toronto,Rosedale


The above output already gives the shape but the assignment says to use the shape method, so here it is...

In [49]:
df.shape

(103, 3)